#Importando Bibliotecas

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

#Tabela


##Base de Dados

In [4]:
URL_BASES = "./../Databases/eICU/"
patient_csv = "patient.csv"
infusiondrug = "infusiondrug.csv"
physicalExam = "physicalExam.csv"
lab = "lab.csv"

url_patient = URL_BASES + patient_csv
url_infusiondrug = URL_BASES + infusiondrug
url_physicalExam = URL_BASES + physicalExam
url_lab = URL_BASES + lab

comeco_intervalo = 60 * 0
final_intervalo = 60 * 24

df_patient = pd.read_csv(url_patient)
df_lab = pd.read_csv(url_lab)
df_physicalExam = pd.read_csv(url_physicalExam)

## Characteristic

In [5]:
characteristic = df_patient [['patientunitstayid', 'gender', 'age', 'ethnicity']].copy()
#Substitua valores específicos por NaN
#characteristic = characteristic.replace(-1, pd.NA)
characteristic.head()

,patientunitstayid,gender,age,ethnicity
0,141168,Female,70,Caucasian
1,141178,Female,52,Caucasian
2,141179,Female,52,Caucasian
3,141194,Male,68,Caucasian
4,141196,Male,71,Caucasian


In [8]:
df_infusiondrug = pd.read_csv("./../Databases/eICU/infusionDrug.csv")

sepsis = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis', case=False, na=False)].reset_index()
sepsis = sepsis[['patientunitstayid','gender','age']]

df_physicalExam = df_physicalExam[(df_physicalExam["physicalexamoffset"]>=comeco_intervalo) &  (df_physicalExam["physicalexamoffset"]<=final_intervalo)]
df_physicalExam = pd.merge(sepsis['patientunitstayid'],df_physicalExam,on='patientunitstayid')

df_lab = df_lab.loc[(df_lab["labresultoffset"] >= comeco_intervalo) & (df_lab["labresultoffset"] <= final_intervalo)]
df_lab = pd.merge(sepsis['patientunitstayid'],df_lab,on='patientunitstayid')

df_infusiondrug = df_infusiondrug[(df_infusiondrug["infusionoffset"]>=comeco_intervalo) &  (df_infusiondrug["infusionoffset"]<=final_intervalo)]
df_infusiondrug = pd.merge(sepsis['patientunitstayid'],df_infusiondrug,on='patientunitstayid')

/tmp/ipykernel_171644/3127796786.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infusiondrug = pd.read_csv("./../Databases/eICU/infusionDrug.csv")


##Organ Dysfunction

Nervous System

In [11]:
sofa = df_patient[['patientunitstayid']].copy()

motor = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
motor = motor[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)

eyes = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
eyes = eyes[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)

verbal = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
verbal = verbal[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)

gcs_temp = pd.merge(motor, eyes, on=['patientunitstayid','physicalexamoffset'], how='outer')
gcs = pd.merge(gcs_temp, verbal, on=['patientunitstayid','physicalexamoffset'], how='outer')
gcs ['Total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

idx = gcs.groupby('patientunitstayid')['Total'].idxmin()
gcs = gcs.drop(['motor', 'eyes', 'verbal'], axis=1)
gcs = gcs.loc[idx]

def score(row):
  if((row['Total']<6)):
    return 4
  elif((row['Total']>=6) and (row['Total']<=9)):
    return 3
  elif((row['Total']>=10) and (row['Total']<=12)):
    return 2
  elif((row['Total']>=13) and (row['Total']<=14)):
    return 1
  else:
    return 0

sofa['Nervous'] = gcs.apply(score, axis=1)
sofa

,patientunitstayid,Nervous
0,141168,NaN
1,141178,NaN
2,141179,NaN
3,141194,1.0
4,141196,NaN
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


Cardiovascular System

In [12]:
dopamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Norepinephrine', case=False, na=False)]

dopamine = dopamine.query('infusionoffset >= 0 and infusionoffset <= 1440')
dobutamine = dobutamine.query('infusionoffset >= 0 and infusionoffset <= 1440')
epinephrine = epinephrine.query('infusionoffset >= 0 and infusionoffset <= 1440')
norepinephrine = norepinephrine.query('infusionoffset >= 0 and infusionoffset <= 1440')

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
dobutamine = dobutamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
epinephrine = epinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
norepinephrine = norepinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()

dopamine['concentration.dop'] = dopamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
epinephrine['concentration.epi'] = epinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
dobutamine['concentration.dob'] = dobutamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
norepinephrine['concentration.nor'] = norepinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)


dopamine.rename(columns={'drugamount' : 'dopamine'}, inplace=True)
dobutamine.rename(columns={'drugamount' : 'dobutamine'}, inplace=True)
epinephrine.rename(columns={'drugamount' : 'epinephrine'}, inplace=True)
norepinephrine.rename(columns={'drugamount' : 'norepinephrine'}, inplace=True)

dopamine = dopamine.groupby('patientunitstayid')['concentration.dop'].max().reset_index()
dobutamine = dobutamine.groupby('patientunitstayid')['concentration.dob'].max().reset_index()
epinephrine = epinephrine.groupby('patientunitstayid')['concentration.epi'].max().reset_index()
norepinephrine = norepinephrine.groupby('patientunitstayid')['concentration.nor'].max().reset_index()

cardio = pd.merge(dopamine, dobutamine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, epinephrine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, norepinephrine, on='patientunitstayid', how='outer')

In [13]:
systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic = systolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
diastolic = diastolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')

systolic = systolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
diastolic = diastolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()

systolic.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic, diastolic, on=['patientunitstayid','physicalexamoffset'])

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

#PAM = (Pressão Sistólica + 2 x Pressão Diastólica) / 3
MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

cardio = pd.merge(cardio, MAP, on='patientunitstayid', how='outer')

In [14]:
def score(row):
  if((row['concentration.dop']>15) or (row['concentration.epi']>0.1) or (row['concentration.nor']>0.1)):
    return 4
  elif((row['concentration.dop']>5) or (row['concentration.epi']<=0.1) or (row['concentration.nor']<=0.1)):
    return 3
  elif((row['concentration.dop']<=5) or (row['concentration.dop']>0)):
    return 2
  elif(row['MAP']<70):
    return 1
  else:
    return 0

sofa['Cardiovascular'] = cardio.apply(score, axis=1)

Respiratory System

In [16]:
pao2 = df_lab[df_lab['labname'].str.contains('^paO2$', case=False, na=False, regex=True)]
fio2 = df_lab[df_lab['labname'].str.contains('^FiO2$', case=False, na=False, regex=True)]

pao2 = pao2.query('labresultoffset >= 0 and labresultoffset <= 1440')
fio2 = fio2.query('labresultoffset >= 0 and labresultoffset <= 1440')

pao2 = pao2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

pao2.rename(columns={'labresult' : 'pao2'}, inplace=True)
fio2.rename(columns={'labresult' : 'fio2'}, inplace=True)

resp = pd.merge(pao2, fio2, on=['patientunitstayid'], how='outer')
resp['Total'] = resp['pao2']/resp['fio2']

def score(row):
    if row['Total'] < 100:
        return 4
    elif 100 <= row['Total'] < 200:
        return 3
    elif 200 <= row['Total'] < 300:
        return 2
    elif 300 <= row['Total'] < 400:
        return 1
    else:
        return 0

resp = resp.groupby('patientunitstayid')['Total'].min().reset_index()
sofa['Respiratory'] = resp.apply(score, axis=1)

Coagulation

In [17]:
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query('labresultoffset >= 0 and labresultoffset <= 1440')
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

def score(row):
    if row['platelets'] < 20:
        return 4
    elif 20 <= row['platelets'] < 50:
        return 3
    elif 50 <= row['platelets'] < 100:
        return 2
    elif 100 <= row['platelets'] < 150:
        return 1
    else:
        return 0

sofa['Coagulation'] = platelets.apply(score, axis=1)

Liver

In [18]:
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]
bilirubin = bilirubin.query('labresultoffset >= 0 and labresultoffset <= 1440')
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

def score(row):
    if row['bilirubin'] >= 12.0:
        return 4
    elif 12.0 > row['bilirubin'] >= 6.0:
        return 3
    elif 6.0 > row['bilirubin'] >= 2.0:
        return 2
    elif 2.0 > row['bilirubin'] > 1.2:
        return 1
    else:
        return 0

sofa['Liver'] = bilirubin.apply(score, axis=1)

Renal Function

In [19]:
creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]
creatinine = creatinine.query('labresultoffset >=0 and labresultoffset <= 1440')
creatinine = creatinine[['patientunitstayid','labresult']].copy()
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

def score(row):
  if row['creatinine'] >= 5.0:
    return 4
  elif 5.0 > row['creatinine'] >= 3.5:
    return 3
  elif 3.0 > row['creatinine'] >= 2.0:
    return 2
  elif 2.0 > row['creatinine'] > 1.2:
    return 1
  else:
    return 0

sofa['Renal'] = creatinine.apply(score, axis=1)

Score Final

In [20]:
sofa['sofascore'] = sofa['Nervous'] + sofa['Cardiovascular'] + sofa['Respiratory'] + sofa['Coagulation'] + sofa['Liver'] + sofa['Renal']
sofa = sofa[['patientunitstayid', 'sofascore']].copy()
sofa

,patientunitstayid,sofascore
0,141168,NaN
1,141178,NaN
2,141179,NaN
3,141194,2.0
4,141196,NaN
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


##Inflammation


In [28]:
bands = df_lab[df_lab['labname'].str.contains('-bands', case=False, na=False)]
CRP = df_lab[df_lab['labname'].str.contains('CRP', case=False, na=False)]
ESR = df_lab[df_lab['labname'].str.contains('ESR', case=False, na=False)]
lactate = df_lab[df_lab['labname'].str.contains('lactate', case=False, na=False)]
temperature = df_lab[df_lab['labname'].str.contains('Temp', case=False, na=False)]
WBC = df_lab[df_lab['labname'].str.contains('WBC x 1000', case=False, na=False)]
#lymphs = df_lab[df_lab['labname'].str.contains('-lymphs', case=False, na=False)]

bands = bands.query('labresultoffset >= 0 and labresultoffset <= 1440')
CRP = CRP.query('labresultoffset >= 0 and labresultoffset <= 1440')
ESR = ESR.query('labresultoffset >= 0 and labresultoffset <= 1440')
temperature = temperature.query('labresultoffset >= 0 and labresultoffset <= 1440')
lactate = lactate.query('labresultoffset >= 0 and labresultoffset <= 1440')
WBC = WBC.query('labresultoffset >= 0 and labresultoffset <= 1440')

bands = bands[['patientunitstayid', 'labresult']].copy()
CRP = CRP[['patientunitstayid', 'labresult']].copy()
ESR = ESR[['patientunitstayid', 'labresult']].copy()
temperature = temperature[['patientunitstayid', 'labresulttext']].copy()
lactate = lactate[['patientunitstayid', 'labresult']].copy()
WBC = WBC[['patientunitstayid', 'labresult']].copy()
#lymphs = lymphs[['patientunitstayid', 'labresult']].copy()

# Transformando as temperaturas em celsius
temperature['temp'] = temperature['labresulttext'].astype(float)
temperature['temp'] = temperature['temp'].map(lambda x: x if x < 60 else (x - 32) * 5/9)

# Pior valor da temperatura
temperature['distancia'] = np.abs(temperature['temp'] - 36.85)
temperature = temperature.loc[temperature.groupby('patientunitstayid')['distancia'].idxmax()]
temperature = temperature.reset_index(drop=True)
temperature = temperature.drop(['distancia', 'labresulttext'], axis=1)

# Calcular o valor mais crítico para WBC
#WBC['distancia'] = np.abs(WBC['labresult'] - 7500)
WBC = WBC.groupby('patientunitstayid')['labresult'].max().reset_index()

# Calcular o valor mais crítico para linfócitos
#lymphs['distancia'] = np.abs(lymphs['labresult'] - 2000)
#lymphs = lymphs.groupby('patientunitstayid')['labresult'].max().reset_index(drop=True)

bands = bands.groupby('patientunitstayid')['labresult'].max().reset_index()
CRP = CRP.groupby('patientunitstayid')['labresult'].max().reset_index()
ESR = ESR.groupby('patientunitstayid')['labresult'].max().reset_index()
lactate= lactate.groupby('patientunitstayid')['labresult'].max().reset_index()

bands = bands.rename(columns={'labresult': 'bands'})
CRP = CRP.rename(columns={'labresult': 'CRP'})
ESR = ESR.rename(columns={'labresult': 'ESR'})
lactate = lactate.rename(columns={'labresult': 'lactate'})
WBC = WBC.rename(columns={'labresult': 'WBC'})
#lymphs = lymphs.rename(columns={'labresult': 'lymphs'})

inflammation = pd.merge(bands, CRP, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, ESR, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, temperature, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, lactate, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, WBC, on='patientunitstayid', how='outer')
#inflammation = pd.merge(inflammation, lymphs, on='patientunitstayid', how='outer')

inflammation

,patientunitstayid,bands,CRP,ESR,temp,lactate,WBC
0,141194,NaN,NaN,NaN,NaN,1.3,14.1
1,141227,10.0,NaN,NaN,NaN,2.1,42.7
2,141266,NaN,NaN,NaN,NaN,NaN,10.0
3,141288,26.0,NaN,NaN,NaN,1.1,16.3
4,141297,20.0,19.7,115.0,NaN,3.0,17.3
...,...,...,...,...,...,...,...
21593,3352884,NaN,NaN,NaN,NaN,1.0,14.4
21594,3352895,NaN,NaN,NaN,NaN,NaN,7.9
21595,3352922,NaN,NaN,NaN,NaN,7.0,19.7
21596,3352940,NaN,NaN,NaN,NaN,4.0,18.9


##Pulmonary

In [29]:
resp = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Resp Current', case=False, na=False)]
resp = resp.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
resp = resp[['patientunitstayid', 'physicalexamtext']].copy()
resp['physicalexamtext'] = resp['physicalexamtext'].astype(float)

resp['distancia'] = np.abs(resp['physicalexamtext'] - 16)
resp = resp.loc[resp.groupby('patientunitstayid')['distancia'].idxmax()]
resp = resp.reset_index(drop=True)
resp = resp.drop(['distancia'], axis=1)

resp = resp.groupby('patientunitstayid')['physicalexamtext'].max().reset_index()
resp = resp.rename(columns={'physicalexamtext': 'resp'})

In [30]:
spo2 = df_lab[df_lab['labname'].str.contains('O2 Sat', case=False, na=False)]
fio2 = df_lab[df_lab['labname'].str.contains('FiO2', case=False, na=False)]

spo2 = spo2.query('labresultoffset >= 0 and labresultoffset <= 1440')
fio2 = fio2.query('labresultoffset >= 0 and labresultoffset <= 1440')

spo2 = spo2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

spo2 = spo2.rename(columns={'labresult': 'spo2'})
fio2 = fio2.rename(columns={'labresult': 'fio2'})

spo2_fio2 = pd.merge(spo2, fio2, on='patientunitstayid')
spo2_fio2['spo2/fio2'] = spo2_fio2['spo2'] / spo2_fio2['fio2']
spo2_fio2 = spo2_fio2.drop(['spo2', 'fio2'], axis=1)
spo2_fio2 = spo2_fio2.groupby('patientunitstayid')['spo2/fio2'].max().reset_index()

In [31]:
pao2 = df_lab[df_lab['labname'].str.contains('paO2', case=False, na=False)]
pao2 = pao2.query('labresultoffset >= 0 and labresultoffset <= 1440')
pao2 = pao2[['patientunitstayid', 'labresult']].copy()
pao2 = pao2.rename(columns={'labresult': 'pao2'})
pao2_fio2 = pd.merge(pao2, fio2, on='patientunitstayid')
pao2_fio2['pao2/fio2'] = pao2_fio2['pao2'] / pao2_fio2['fio2']
pao2_fio2 = pao2_fio2.drop(['pao2', 'fio2'], axis=1)
pao2_fio2 = pao2_fio2.groupby('patientunitstayid')['pao2/fio2'].min().reset_index()

In [32]:
pulmonary = pd.merge(resp, spo2_fio2, on='patientunitstayid', how='outer')
pulmonary = pd.merge(pulmonary, pao2_fio2, on='patientunitstayid', how='outer')
pulmonary

,patientunitstayid,resp,spo2/fio2,pao2/fio2
0,141194,25.0,NaN,NaN
1,141197,24.0,NaN,NaN
2,141227,NaN,NaN,0.650000
3,141266,19.0,NaN,NaN
4,141288,NaN,NaN,0.810000
...,...,...,...,...
18876,3352721,23.0,3.333333,1.170000
18877,3352895,12.0,3.233333,1.333333
18878,3352922,19.0,2.000000,3.640000
18879,3352940,55.0,0.938776,0.693878


##Cardiovascular

In [33]:
bicarbonate = df_lab[df_lab['labname'].str.contains('bicarbonate', case=False, na=False)]
bicarbonate = bicarbonate.query('labresultoffset >= 0 and labresultoffset <= 1440')
bicarbonate = bicarbonate[['patientunitstayid', 'labresult']].copy()
bicarbonate.rename(columns={'labresult': 'bicarbonate'}, inplace=True)
bicarbonate = bicarbonate.groupby('patientunitstayid')['bicarbonate'].min().reset_index()

In [34]:
heartRate = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains('HR Current', case=False, na=False)]
heartRate = heartRate.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
heartRate = heartRate[['patientunitstayid', 'physicalexamtext']].copy()

heartRate['physicalexamtext'] = heartRate['physicalexamtext'].astype(float)
heartRate['distancia'] = np.abs(heartRate['physicalexamtext'] - 75)
heartRate = heartRate.loc[heartRate.groupby('patientunitstayid')['distancia'].idxmax()]
heartRate = heartRate.reset_index(drop=True)
heartRate = heartRate.drop(['distancia'], axis=1)

heartRate.rename(columns={'physicalexamtext': 'heartRate'}, inplace=True)

In [41]:
systolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains('systolic', case=False, na=False)]
systolic = systolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()

#systolic['physicalexamtext'] = systolic['physicalexamtext'].map(lambda x: 0 if x=='systolic murmur' else int(x))

#systolic['distancia'] = np.abs(systolic['physicalexamtext'] - 115)
systolic = systolic.loc[systolic.groupby('patientunitstayid')['physicalexamtext'].idxmax()]
systolic = systolic.drop_duplicates(subset='patientunitstayid')
systolic = systolic.reset_index(drop=True)
#systolic = systolic.drop(['distancia'], axis=1)

systolic.rename(columns={'physicalexamtext': 'systolic'}, inplace=True)

In [42]:
troponin = df_lab[df_lab['labname'].str.contains('troponin', case=False, na=False)]
troponin = troponin.query('labresultoffset >= 0 and labresultoffset <= 1440')
troponin = troponin[['patientunitstayid', 'labresult']].copy()
troponin.rename(columns={'labresult': 'troponin'}, inplace=True)
troponin = troponin.groupby('patientunitstayid')['troponin'].max().reset_index()

In [46]:
diastolic.sort_values(by='physicalexamtext').head(5000000000)

,patientunitstayid,physicalexamtext
194314,996829,-1
428570,2600061,-1
428603,2600061,-1
44677,290460,-1
44650,290460,-1
...,...,...
583442,3095938,diastolic murmur
623761,3147673,diastolic murmur
623697,3147673,diastolic murmur
76578,452561,diastolic murmur


In [43]:
diastolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains('diastolic', case=False, na=False)]
diastolic = diastolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
diastolic = diastolic[['patientunitstayid', 'physicalexamtext']].copy()
diastolic['physicalexamtext'] = diastolic['physicalexamtext'].astype(float)

diastolic['distancia'] = np.abs(diastolic['physicalexamtext'] - 62.5)
diastolic = diastolic.loc[diastolic.groupby('patientunitstayid')['distancia'].idxmax()]
diastolic = diastolic.drop_duplicates(subset='patientunitstayid')
diastolic = diastolic.reset_index(drop=True)
diastolic = diastolic.drop(['distancia'], axis=1)

diastolic.rename(columns={'physicalexamtext': 'diastolic'}, inplace=True)

ValueError: could not convert string to float: 'diastolic murmur'

In [ ]:
systolic_map = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic_map = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic_map = systolic_map.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
diastolic_map = diastolic_map.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')

systolic_map = systolic_map[['patientunitstayid', 'physicalexamtext']].copy()
diastolic_map = diastolic_map[['patientunitstayid', 'physicalexamtext']].copy()

systolic_map.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic_map.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic_map, diastolic_map, on='patientunitstayid')

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

In [ ]:
cardiovascular = pd.merge(bicarbonate, heartRate, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, troponin, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, systolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, diastolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, MAP, on='patientunitstayid', how='outer')
cardiovascular

##Renal

In [ ]:
BUN = df_lab[df_lab['labname'].str.contains('BUN', case=False, na=False)]
BUN = BUN.query('labresultoffset >= 0 and labresultoffset <= 1440')
BUN = BUN[['patientunitstayid', 'labresult']].copy()
BUN.rename(columns={'labresult' : 'bloodUreaNitrogen'}, inplace=True)
BUN = BUN.groupby('patientunitstayid')['bloodUreaNitrogen'].max().reset_index()
renal = BUN

renal

##Hepatic

In [ ]:
# Selecionar indicadores hepáticos no dataset 'lab'
alanine = df_lab[df_lab['labname'].str.contains('ALT', case=False, na=False)]
aspartate = df_lab[df_lab['labname'].str.contains('AST', case=False, na=False)]
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]

# Filtrar para o intervalo das primeiras 24 horas do paciente na UTI
alanine = alanine.query('labresultoffset >= 0 and labresultoffset <= 1440')
aspartate = aspartate.query('labresultoffset >= 0 and labresultoffset <= 1440')
bilirubin = bilirubin.query('labresultoffset >= 0 and labresultoffset <= 1440')

# Fazer uma cópia com as colunas importantes para o modelo
alanine = alanine[['patientunitstayid', 'labresult']].copy()
aspartate = aspartate[['patientunitstayid', 'labresult']].copy()
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()

# Renomear colunas para facilitar visualização
alanine.rename(columns={'labresult' : 'alanine'}, inplace=True)
aspartate.rename(columns={'labresult' : 'aspartate'}, inplace=True)
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)

# Filtrar os valores mais críticos de cada paciente
alanine = alanine.groupby('patientunitstayid')['alanine'].max().reset_index()
aspartate = aspartate.groupby('patientunitstayid')['aspartate'].max().reset_index()
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

# Mesclar os datasets
hepatic = pd.merge(alanine, aspartate, on='patientunitstayid', how='outer')
hepatic = pd.merge(hepatic, bilirubin, on='patientunitstayid', how='outer')

hepatic

##Hematologic

In [ ]:
hemoglobin = df_lab[df_lab['labname'].str.contains('Hgb', case=False, na=False)]
hemoglobin = hemoglobin.query('labresultoffset >= 0 and labresultoffset <= 1440')
hemoglobin = hemoglobin[['patientunitstayid', 'labresult']].copy()
hemoglobin.rename(columns={'labresult' : 'hemoglobin'}, inplace=True)
hemoglobin = hemoglobin.groupby('patientunitstayid')['hemoglobin'].min().reset_index()

In [ ]:
iqr = df_lab[df_lab['labname'].str.contains('INR', case=False, na=False)]
iqr = iqr.query('labresultoffset >= 0 and labresultoffset <= 1440')
iqr = iqr[['patientunitstayid', 'labresult']].copy()
iqr.rename(columns={'labresult' : 'iqr'}, inplace=True)
iqr = iqr.groupby('patientunitstayid')['iqr'].max().reset_index()

In [ ]:
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query('labresultoffset >= 0 and labresultoffset <= 1440')
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

In [ ]:
PTT = df_lab[df_lab['labname'].str.contains('PTT', case=False, na=False)]
PTT = PTT.query('labresultoffset >= 0 and labresultoffset <= 1440')
PTT = PTT[['patientunitstayid', 'labresult']].copy()
PTT.rename(columns={'labresult' : 'PTT'}, inplace=True)
PTT = PTT.groupby('patientunitstayid')['PTT'].max().reset_index()

In [ ]:
fibrinogen = df_lab[df_lab['labname'].str.contains('fibrinogen', case=False, na=False)]
fibrinogen = fibrinogen.query('labresultoffset >= 0 and labresultoffset <= 1440')
fibrinogen = fibrinogen[['patientunitstayid', 'labresult']].copy()

fibrinogen['distancia'] = np.abs(fibrinogen['labresult'] - 300)
fibrinogen = fibrinogen.dropna(subset=['distancia'])
fibrinogen = fibrinogen.loc[fibrinogen.groupby('patientunitstayid')['distancia'].idxmax()]
fibrinogen = fibrinogen.reset_index(drop=True)
fibrinogen = fibrinogen.drop(['distancia'], axis=1)

fibrinogen.rename(columns={'labresult' : 'fibrinogen'}, inplace=True)

In [ ]:
hematologic = pd.merge(hemoglobin, iqr, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, platelets, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, PTT, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, fibrinogen, on='patientunitstayid', how='outer')
hematologic

##Bioquímica

In [ ]:
albumin = df_lab[df_lab['labname'].str.contains('albumin', case=False, na=False)]
albumin = albumin.query('labresultoffset >= 0 and labresultoffset < 1440')
albumin = albumin[['patientunitstayid', 'labresult']].copy()
albumin.rename(columns={'labresult' : 'albumin'}, inplace=True)
albumin = albumin.groupby('patientunitstayid')['albumin'].min().reset_index()

In [ ]:
chloride = df_lab[df_lab['labname'].str.contains('chloride', case=False, na=False)]
chloride = chloride.query('labresultoffset >= 0 and labresultoffset < 1440')
chloride = chloride[['patientunitstayid', 'labresult']].copy()

chloride['distancia'] = np.abs(chloride['labresult'] - 102.5)
chloride = chloride.dropna(subset=['distancia'])
chloride = chloride.loc[chloride.groupby('patientunitstayid')['distancia'].idxmax()]
chloride = chloride.reset_index(drop=True)
chloride = chloride.drop(['distancia'], axis=1)

chloride.rename(columns={'labresult' : 'chloride'}, inplace=True)

In [ ]:
glucose = df_lab[df_lab['labname'].str.contains('glucose', case=False, na=False)]
glucose = glucose.query('labresultoffset >= 0 and labresultoffset < 1440')
glucose = glucose[['patientunitstayid', 'labresult']].copy()

glucose['distancia'] = np.abs(glucose['labresult'] - 125)
glucose = glucose.dropna(subset=['distancia'])
glucose = glucose.loc[glucose.groupby('patientunitstayid')['distancia'].idxmax()]
glucose = glucose.reset_index(drop=True)
glucose = glucose.drop(['distancia'], axis=1)

glucose.rename(columns={'labresult' : 'glucose'}, inplace=True)

In [ ]:
sodium = df_lab[df_lab['labname'].str.contains('sodium', case=False, na=False)]
sodium = sodium.query('labresultoffset >= 0 and labresultoffset < 1440')
sodium = sodium[['patientunitstayid', 'labresult']].copy()

sodium['distancia'] = np.abs(sodium['labresult'] - 140)
sodium = sodium.dropna(subset=['distancia'])
sodium = sodium.loc[sodium.groupby('patientunitstayid')['distancia'].idxmax()]
sodium = sodium.reset_index(drop=True)
sodium = sodium.drop(['distancia'], axis=1)

sodium.rename(columns={'labresult' : 'sodium'}, inplace=True)

In [ ]:
motor = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
motor = motor[['patientunitstayid', 'physicalexamtext']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)

eyes = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
eyes = eyes[['patientunitstayid', 'physicalexamtext']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)

verbal = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal.query('physicalexamoffset >= 0 and physicalexamoffset < 1440')
verbal = verbal[['patientunitstayid', 'physicalexamtext']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)

gcs_temp = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
gcs = pd.merge(gcs_temp, verbal, on='patientunitstayid', how='outer')
gcs ['gcs total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

idx = gcs.groupby('patientunitstayid')['gcs total'].idxmin()
gcs = gcs.drop(['motor', 'eyes', 'verbal'], axis=1)
gcs = gcs.loc[idx]

In [ ]:
potassium = df_lab[df_lab['labname'].str.contains('potassium', case=False, na=False)]
potassium = potassium.query('labresultoffset >= 0 and labresultoffset < 1440')
potassium = potassium[['patientunitstayid', 'labresult']].copy()

potassium['distancia'] = np.abs(potassium['labresult'] - 4.5)
potassium = potassium.dropna(subset=['distancia'])
potassium = potassium.loc[potassium.groupby('patientunitstayid')['distancia'].idxmax()]
potassium = potassium.reset_index(drop=True)
potassium = potassium.drop(['distancia'], axis=1)

potassium.rename(columns={'labresult' : 'potassium'}, inplace=True)

In [ ]:
calcium = df_lab[df_lab['labname'].str.contains('calcium', case=False, na=False)]
calcium = calcium.query('labresultoffset >= 0 and labresultoffset < 1440')
calcium = calcium[['patientunitstayid', 'labresult']].copy()

calcium['distancia'] = np.abs(calcium['labresult'] - 9.6)
calcium = calcium.dropna(subset=['distancia'])
calcium = calcium.loc[calcium.groupby('patientunitstayid')['distancia'].idxmax()]
calcium = calcium.reset_index(drop=True)
calcium = calcium.drop(['distancia'], axis=1)

calcium.rename(columns={'labresult' : 'calcium'}, inplace=True)

In [ ]:
phosphate = df_lab[df_lab['labname'].str.contains('phosphate', case=False, na=False)]
phosphate = phosphate.query('labresultoffset >= 0 and labresultoffset < 1440')
phosphate = phosphate[['patientunitstayid', 'labresult']].copy()

phosphate['distancia'] = np.abs(phosphate['labresult'] - 3.5)
phosphate = phosphate.dropna(subset=['distancia'])
phosphate = phosphate.loc[phosphate.groupby('patientunitstayid')['distancia'].idxmax()]
phosphate = phosphate.reset_index(drop=True)
phosphate = phosphate.drop(['distancia'], axis=1)

phosphate.rename(columns={'labresult' : 'phosphate'}, inplace=True)

In [ ]:
magnesium = df_lab[df_lab['labname'].str.contains('magnesium', case=False, na=False)]
magnesium = magnesium.query('labresultoffset >= 0 and labresultoffset < 1440')
magnesium = magnesium[['patientunitstayid', 'labresult']].copy()
magnesium.rename(columns={'labresult' : 'magnesium'}, inplace=True)
magnesium = magnesium.groupby('patientunitstayid')['magnesium'].min().reset_index()

In [ ]:
bioquimica = pd.merge(albumin, chloride, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, glucose, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, sodium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, gcs, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, potassium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, calcium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, phosphate, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, magnesium, on='patientunitstayid', how='outer')
bioquimica

##Merge Set

In [ ]:
# Internações: 0 por outro diagnóstico e 1 para Sepse
diagnostico = df_diagnosis [['patientunitstayid', 'diagnosisstring']].copy()

# Criar a coluna de diagnóstico de sepse ou não
diagnostico['sepse_diagnostico'] = 0

# Verificar se ocorre a palavra "sepse" na coluna 'apacheadmissiondx' e atribuir 1 se ocorrer
diagnostico.loc[diagnostico['diagnosisstring'].str.contains('sepsis', case=False, na=False), 'sepse_diagnostico'] = 1
diagnostico = diagnostico[['patientunitstayid','sepse_diagnostico']].copy()
diagnostico = diagnostico.groupby('patientunitstayid')['sepse_diagnostico'].max().reset_index()

# Juntar as tabelas para avaliar o SOFA
diagnostico = pd.merge(diagnostico, sofa, on='patientunitstayid', how='outer')
diagnostico['sofa_diagnostico'] = diagnostico['sofascore'].map(lambda x: 1 if x >= 2 else 0)

# Função para juntar os dois diagnósticos
def issepsis(row):
  if row['sepse_diagnostico']==1 or row['sofa_diagnostico']==1:
    return 1
  else:
    return 0

diagnostico['diagnostico'] = diagnostico.apply(issepsis, axis=1)
diagnostico = diagnostico.drop(['sepse_diagnostico', 'sofascore', 'sofa_diagnostico'], axis=1)
diagnostico

In [ ]:
df_final = pd.merge(characteristic, sofa, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, inflammation, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, pulmonary, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, cardiovascular, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, renal, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hepatic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hematologic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, bioquimica, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, diagnostico, on='patientunitstayid', how='outer')
df_final

## Numerical Values

In [ ]:
df_final['age'] = df_final['age'].map(lambda x: '95' if x == '> 89' else x)
df_final['age'] = df_final['age'].astype(float)

In [ ]:
df_final['gender'] = df_final['gender'].map(lambda x: 0 if x == 'Male' else 1)
df_final['gender'] = df_final['gender'].astype(int)

In [ ]:
def ethn(x):
  if(x=='Caucasian'):
    return 1
  elif(x=='Hispanic'):
    return 2
  elif(x=='Asian'):
    return 3
  elif(x=='African American'):
    return 4
  elif(x=='Native American'):
    return 5
  else:
    return 0

df_final['ethnicity'] = df_final['ethnicity'].apply(ethn)
df_final['ethnicity'].unique()

## Missing Data

In [ ]:
df_numeric = df_final.apply(lambda col: pd.to_numeric(col, errors='coerce'))
col_stats = {
    'max': df_numeric.max(skipna=True),
    'min': df_numeric.min(skipna=True),
    'nulos': df_final.isnull().sum(),
    'percentual_nulos': df_final.isnull().mean() * 100,
}

# Criar um DataFrame a partir das estatísticas
df_col_stats = pd.DataFrame(col_stats)
df_col_stats['percentual_nulos'] = df_col_stats['percentual_nulos'].apply(lambda x: '{:.2f}%'.format(x))

# Ajustar o índice para ter os nomes das colunas do 'sepse' como linhas do novo DataFrame
df_col_stats.index.name = 'Colunas'

df_col_stats.sort_values(by = "percentual_nulos",ascending=False)

### Dealing with Missing Data

In [ ]:
'''age_n = df_final['age'].mean(axis=0)

df_final['gender'] = df_final['gender'].fillna(0)
df_final['age'] = df_final['age'].fillna(age_n)
df_final['sofascore'] = df_final['sofascore'].fillna(0)'''

In [ ]:
#Inflammation
'''
df_final['bands'] = df_final['bands'].fillna(250)
df_final['CRP'] = df_final['CRP'].fillna(5)

df_final['ESR'] = df_final['ESR'].fillna(10)
df_final['temp'] = df_final['temp'].fillna(36.85)
df_final['lactate'] = df_final['lactate'].fillna(1)
df_final['WBC'] = df_final['WBC'].fillna(7500)
df_final['lymphs'] = df_final['lymphs'].fillna(2000)

#Pulmonary
df_final['resp'] = df_final['resp'].fillna(16)
df_final['spo2/fio2'] = df_final['spo2/fio2'].fillna(450)
df_final['pao2/fio2'] = df_final['pao2/fio2'].fillna(600)

#Cardiovascular
df_final['bicarbonate'] = df_final['bicarbonate'].fillna(23)
df_final['heartRate'] = df_final['heartRate'].fillna(75)
df_final['troponin'] = df_final['troponin'].fillna(115)
df_final['systolic'] = df_final['systolic'].fillna(0.01)
df_final['diastolic'] = df_final['diastolic'].fillna(62.5)
df_final['MAP'] = df_final['MAP'].fillna(85)

#Renal
df_final['bloodUreaNitrogen'] = df_final['bloodUreaNitrogen'].fillna(10)

#Hepatic
df_final['alanine'] = df_final['alanine'].fillna(24)
df_final['aspartate'] = df_final['aspartate'].fillna(22.5)
df_final['bilirubin'] = df_final['bilirubin'].fillna(0.6)

#Hematologic
df_final['hemoglobin'] = df_final['hemoglobin'].fillna(18)
df_final['iqr'] = df_final['iqr'].fillna(0.6)
df_final['platelets'] = df_final['platelets'].fillna(275)
df_final['PTT'] = df_final['PTT'].fillna(0.8)
df_final['fibrinogen'] = df_final['fibrinogen'].fillna(300)

#Bioquimica
df_final['albumin'] = df_final['albumin'].fillna(4.35)
df_final['chloride'] = df_final['chloride'].fillna(102.5)
df_final['glucose'] = df_final['glucose'].fillna(125)
df_final['sodium'] = df_final['sodium'].fillna(140)
df_final['gcs total'] = df_final['gcs total'].fillna(15)
df_final['potassium'] = df_final['potassium'].fillna(4.5)
df_final['calcium'] = df_final['calcium'].fillna(9.6)
df_final['phosphate'] = df_final['phosphate'].fillna(3.5)
df_final['magnesium'] = df_final['magnesium'].fillna(2.1)


#Diagnostico
df_final['diagnostico'] = df_final['diagnostico'].fillna(0)

#df_final.isna().sum()'''